In [1]:
from typing import List
import matplotlib.pyplot as plt
import random
import copy
import numpy as np
class Position:
    def __init__(self, x,y):
        self.x = x
        self.y=y
class Elf(Position):
    def step(self,direction:str):
        self.prev = (self.x, self.y)
        if direction =="up":
            self.x += -1
        elif direction =="down":
            self.x += 1
        elif direction =="left":
            self.y += -1
        elif direction =="right":
            self.y += 1
    def backup(self):
        self.x, self.y = self.prev
class Environment:
    def __init__(self, entrance=None,file_path="day24/input.txt", elf =None):
        with open(file_path,"r") as file:
            lines = file.readlines()
        raw_data = [line.strip() for line in lines]
        raw_temp =[]
        for line in raw_data:
            temp = []
            for element in line:
                temp.append(element)
            raw_temp.append(temp)
        raw_data = np.array(raw_temp)
        inner_data = raw_data[1:raw_data.shape[0]-1,1:raw_data.shape[1]-1]
        outer_data = np.where(raw_data=="#", 1, 0 )
        if not elf:
            self.elf = Elf(0, 1)
        else:
            self.elf = Elf(elf[0], elf[1])
        outer_data[self.elf.x,self.elf.y] = 0

        if not entrance:
            self.entrance = outer_data.shape[0]-1,outer_data.shape[1]-2

        else:
            self.entrance = entrance
        outer_data[self.entrance[0],self.entrance[1]] =-1

        self.outer_data = outer_data
        self.left_data = np.where(inner_data=="<", 2, 0 )
        self.right_data = np.where(inner_data==">",4, 0)
        self.up_data = np.where(inner_data=="^",3, 0)
        self.down_data = np.where(inner_data=="v",5, 0)
        self.time =0
        self.game_status = "active"
    def check_status(self):
        data = self.get_data()
        if (self.elf.x, self.elf.y) == self.entrance:
            return 1
        if self.elf.x >= data.shape[0]-1 or self.elf.y >= data.shape[1]-1 or self.elf.x <0 or self.elf.y <0:
            return -1
        value = data[self.elf.x][self.elf.y]
        if value == -1:
            return 1
        elif value ==0:
            return 0
        else: return -1
        
    def get_data(self):
        re_inner_data = self.left_data+self.right_data+self.up_data + self.down_data
        re_inner_data = np.pad(re_inner_data,1,constant_values=0)
        return re_inner_data + self.outer_data
    def progress_env(self, n_steps, reset_state=False):
        # print("progress received n_steps ", n_steps, "reset_state ", reset_state, " time ", self.time )


        if not reset_state:
            self.time += n_steps
            roll_steps = n_steps
        else:
            roll_steps = n_steps- self.time
            self.time = n_steps
        self.left_data =np.roll(self.left_data,-roll_steps,axis =1)
        self.right_data =np.roll(self.right_data,roll_steps,axis =1)
        self.up_data =np.roll(self.up_data,-roll_steps,axis =0)
        self.down_data =np.roll(self.down_data,roll_steps,axis =0)
        # assert self.check_status() >=0, str(self.elf.x) + " " + str(self.elf.y) + " " + str(hash(str(env.get_data()))) + " time " + str(self.time)

    def backup(self):

        self.elf.backup()
        # print("call progress_env at backup")
        self.progress_env(-1)

    def step(self,direction) -> int:
        # print("call progress at step")
        self.progress_env(1)
        self.elf.step(direction)
        if self.check_status() <0:
            self.game_status = "over"
            return -1
        elif self.check_status() >0:
            self.game_status = "done"
            return 1
        else: 
            self.game_status = "active"
            return 0
    def printout(self):
        data = self.get_data()
        i=0
        for row in data:
            j=0
            print()
            for item in row:
                if (i,j) == (self.elf.x,self.elf.y):
                    if item!=0:
                        print("X", end="")
                    else:
                        print("E", end="")
                else:
                    if item==1:
                        print('#', end="")
                    elif item==0 or item ==-1:
                        print(".", end="")
                    elif item == 2:
                        print("<", end="")
                    elif item == 4:
                        print(">", end="")
                    elif item == 3:
                        print("^", end="")
                    elif item == 5:
                        print("v", end="")
                    else:
                        print("&", end="")
                j += 1
            i += 1

    def clone(self):
        env = copy.deepcopy(self)
        return env


Dijkstras implementation

In [8]:
import queue

# Breadth First Search #
# visited_set
# parent_map: {node: (parent, actions)}
# Queue q
# starting position
# v = starting position
# q.enqueue(starting_position)
# while q not empty():
# v = q.dequeue()
# if v is the goal then return v
# adjancent_nodes, actions = get_adjancent_nodes(v)
# for each node in adjancent_nodes:
    # if node is not in visited_set:
        #q.enqueue(node)
        # node.parent = v
        # node.parent_actions = actions
        # visited_set.add(node)
def test_action(action, env):
        reward = env.step(action)
        status = env.check_status()
        x,y, time = env.elf.x, env.elf.y, env.time
        env_state = hash(str(env.get_data()))
        env.backup()

        return reward >= 0, (x,y,env_state, time)
def get_adjacents(node, env):
    
    output=[]
    ok_actions =[]
    env_hash = hash(str(env.get_data()))
    node_id = (node[0], node[1], env_hash)
    for action in ["wait", "up","down", "left", "right"]:
        result = test_action(action, env)
        if result[0]:
            output.append((result[1][0],result[1][1],result[1][2],result[1][3],[action]))
            ok_actions.append(action)
    waits=[]
    wait_count =0
    env.progress_env(-wait_count) #reverse environment back

    return output
def find_shortest_path(file_path="day24/input.txt",starting_time=0, starting_position=(0,1), entrance = (21,150)):
    dead_nodes = []
    visited_nodes = set()

    q = queue.Queue()


    env = Environment(file_path=file_path,entrance=entrance, elf=starting_position)
    env.progress_env(starting_time,)

    data = env.get_data()


    parent_map = {}
    env_hash = hash(str(env.get_data()))
    starting_node = ((starting_position[0], starting_position[1], env_hash),starting_time)
    q.put(starting_node)
    solutions = []
    while not q.empty():
        node = q.get()
        # print(node)
        if (node[0][0], node[0][1]) == entrance:
            ##handle retrieving
            print("solution found")
            actions = []
            node_id = (node[0][0], node[0][1], node[0][2])
            while True:
                parent = parent_map.get(node_id,"")
                if parent !="":
                    node_id = parent[0]
                    actions = parent[1] + actions
                    if node_id == (starting_position[0], starting_position[1], env_hash):
                        break
            solutions.append(actions)
        

        
        env.elf.x, env.elf.y = node[0][0],node[0][1] #update elf position for environment
        time = node[1]
        env.progress_env(time) #update the environment progress
        assert env.check_status() >= 0, "time " + str(time)  + " x, y " + str(env.elf.x) + " " + str(env.elf.y)
        adjacent_items = get_adjacents(node, env)

        for adjacent_item in adjacent_items:
            adjacent_node = (adjacent_item[0], adjacent_item[1],adjacent_item[2])
            if adjacent_node not in visited_nodes:
                visited_nodes.add(adjacent_node)
                q.put((adjacent_node,adjacent_item[3]))
                parent_map[adjacent_node]= (node[0],adjacent_item[4])
    if len(solutions)>0:
        solutions.sort(key = lambda x: len(x))
        # print("actions ", solutions[0])
        print("time ", len(solutions[0]))
        return len(solutions[0])
    else:
        print("solution not found")
# stage1 = find_shortest_path(starting_time=0, starting_position=(0,1), entrance = (21,150))
# print("stage 1 takes ",stage1)


In [9]:
stage2 = find_shortest_path(starting_time=332, starting_position=(21,150), entrance = (0,1))
print("stage 2 takes ",stage2)


AssertionError: time 332 x, y 21 150

In [ ]:
stage3 = find_shortest_path(starting_time=332+stage2, starting_position=(0,1), entrance = (21,150))
print("stage 3 takes ",stage3)
print("total time for 3 trips ", 332+stage2+stage3)

In [1]:
import dataclasses
import heapq
import time
from typing import Any, TypeAlias

Point: TypeAlias = tuple[int, int]  # x, y. top-left is 0, 0.
Blizzard: TypeAlias = tuple[Point, int]  # starting position, direction (0, 1, 2, 3 => north, east, south west)


DELTAS = ((0, -1), (1, 0), (0, 1), (-1, 0))  # north, east, south, west


def read_input_file(
    file_name: str = "input.txt",
) -> tuple[Point, Point, set[Blizzard], int, int]:  # start, end, blizzards, width, height
    with open(file_name, "r") as f:
        start: Point | None = None
        end: Point | None = None
        blizzards: set[Blizzard] = set()
        for y, line in enumerate(f.readlines()):
            if stripped_line := line.strip():
                if (len(stripped_line) - 1) == stripped_line.count("#"):
                    if y == 0:
                        assert start is None
                        start = (stripped_line.index("."), y)
                    else:
                        assert end is None
                        end = (stripped_line.index("."), y)
                else:
                    blizzards |= {
                        ((x, y), {"^": 0, ">": 1, "v": 2, "<": 3}[ch])
                        for x, ch in enumerate(stripped_line)
                        if ch in ["^", ">", "v", "<"]
                    }
        assert start is not None and end is not None
        return start, end, blizzards, len(stripped_line), y + 1


def heuristic(source: Point, target: Point) -> int:
    # a simple admissible heuristic for the cost of moving from `source` to `target` - Manhattan distance :)
    return abs(source[0] - target[0]) + abs(source[1] - target[1])


def add_points(point_a: Point, point_b: Point) -> Point:
    return point_a[0] + point_b[0], point_a[1] + point_b[1]


def apply_magnitude_to_point(point: Point, magnitude: int) -> Point:
    return point[0] * magnitude, point[1] * magnitude


@dataclasses.dataclass(frozen=True)
class State:
    h: int = dataclasses.field()  # cached heuristic
    point: Point = dataclasses.field()
    time: int = dataclasses.field()

    # defining these for heapq usage
    def __eq__(self, other: Any) -> bool:
        return isinstance(other, State) and self.h == other.h and self.time == other.time

    def __lt__(self, other: Any) -> bool:
        return isinstance(other, State) and self.time < other.time


def a_star_search(
    start: Point, target: Point, blizzards: set[Blizzard], width: int, height: int, start_time: int = 0
) -> list[State]:
    # TODO: could reuse `computed_blizzard_movements` between `a_star_search` calls. don't really feel like it atm tho.
    computed_blizzard_movements: dict[int, set[Point]] = {}  # where the blizzard will be at some time
    previous_points: dict[State, State] = {}

    def is_point_within_map(point: Point) -> bool:
        return (0 < point[0] < (width - 1) and 0 < point[1] < (height - 1)) or point in [start, target]

    def move_blizzard(point: Point, direction: int, time_steps: int) -> Point:
        # calculate how far the blizzard moves between t=0 and t=`time_steps`, then clamp it to the allowable board area
        new_point = add_points(point, apply_magnitude_to_point(DELTAS[direction], time_steps))
        clamped_point = ((new_point[0] - 1) % (width - 2) + 1), ((new_point[1] - 1) % (height - 2) + 1)
        return clamped_point

    def possible_moves(state: State) -> list[State]:
        new_time = state.time + 1
        if (blizzard_positions_at_time := computed_blizzard_movements.get(new_time, None)) is None:
            # blizzards have not yet been computed for this time step - compute and store them
            blizzard_positions_at_time = {
                move_blizzard(point=blizzard_point, direction=blizzard_direction, time_steps=new_time)
                for (blizzard_point, blizzard_direction) in blizzards
            }
            computed_blizzard_movements[new_time] = blizzard_positions_at_time

        return [
            State(point=p, time=new_time, h=heuristic(p, target))
            for p in [*[add_points(state.point, delta) for delta in DELTAS], state.point]
            if is_point_within_map(p) and p not in blizzard_positions_at_time
        ]

    def reconstruct_path(point: State) -> list[State]:
        if point in previous_points.keys():
            return [point] + reconstruct_path(previous_points[point])
        return [point]

    frontier: list[State] = []
    start_state = State(point=start, h=heuristic(start, target), time=start_time)
    heapq.heappush(frontier, start_state)
    g_score: dict[State, int] = {start_state: 0}
    while True:
        if not frontier:
            return []  # infeasible
        current_state = heapq.heappop(frontier)
        if current_state.point == target:
            return reconstruct_path(current_state)
        moves = possible_moves(current_state)
        for move in moves:
            tentative_score = g_score.get(current_state, 1_000_000) + (move.time - current_state.time)
            if tentative_score < g_score.get(move, 1_000_000):
                previous_points[move] = current_state
                g_score[move] = tentative_score
                heapq.heappush(frontier, move)


t0 = time.time()
s, e, b, w, h = read_input_file("day24/input.txt")
leg_1 = a_star_search(start=s, target=e, blizzards=b, width=w, height=h)
if leg_1:
    print(f"Fastest time to reach the goal: {leg_1[0].time} mins.")
    leg_2 = a_star_search(start=e, target=s, blizzards=b, width=w, height=h, start_time=leg_1[0].time)
    assert len(leg_2) > 0
    leg_3 = a_star_search(start=s, target=e, blizzards=b, width=w, height=h, start_time=leg_2[0].time)
    assert len(leg_3) > 0
    print(f"Fastest time to reach the goal, then go back to the start and to the goal again: {leg_3[0].time} mins.")
else:
    print("Problem is infeasible.")
print(f"Solve completed in {round(time.time() - t0, 2)} seconds.")

Fastest time to reach the goal: 332 mins.
Fastest time to reach the goal, then go back to the start and to the goal again: 942 mins.
Solve completed in 8.43 seconds.


In [23]:
actions =['wait', 'wait', 'wait', 'wait', 'down', 'down', 'wait', 'wait', 'wait', 'down', 'down', 'right', 'right', 'down', 'right', 'wait', 'wait', 'right', 'wait', 'up', 'up', 'down', 'wait', 'down', 'right', 'down', 'down', 'right', 'right', 'right', 'wait', 'up', 'right', 'wait', 'up', 'right', 'down', 'right', 'up', 'down', 'down', 'right', 'right', 'wait', 'wait', 'up', 'down', 'down', 'down', 'down', 'left', 'wait', 'wait', 'right', 'right', 'right', 'right', 'right', 'right', 'wait', 'down', 'down', 'right', 'right', 'wait', 'wait', 'up', 'right', 'right', 'right', 'wait', 'up', 'wait', 'wait', 'wait', 'wait', 'up', 'down', 'right', 'right', 'right', 'right', 'right', 'up', 'up', 'right', 'down', 'right', 'right', 'wait', 'left', 'right', 'right', 'right', 'up', 'up', 'right', 'up', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'wait', 'up', 'left', 'down', 'right', 'right', 'down', 'right', 'down', 'wait', 'wait', 'wait', 'right', 'right', 'right', 'up', 'right', 'right', 'right', 'down', 'right', 'right', 'wait', 'up', 'right', 'right', 'right', 'up', 'right', 'up', 'right', 'wait', 'right', 'right', 'right', 'right', 'right', 'down', 'down', 'right', 'down', 'wait', 'up', 'right', 'right', 'right', 'down', 'right', 'right', 'wait', 'down', 'right', 'down', 'right', 'down', 'down', 'wait', 'wait', 'down', 'down', 'down', 'right', 'right', 'right', 'wait', 'wait', 'right', 'up', 'up', 'right', 'right', 'down', 'right', 'up', 'right', 'down', 'right', 'wait', 'wait', 'wait', 'down', 'down', 'right', 'right', 'right', 'up', 'right', 'down', 'right', 'up', 'right', 'right', 'wait', 'up', 'right', 'wait', 'left', 'right', 'up', 'up', 'down', 'right', 'right', 'up', 'right', 'right', 'right', 'right', 'down', 'right', 'down', 'down', 'right', 'down', 'up', 'wait', 'wait', 'down', 'down', 'wait', 'wait', 'wait', 'up', 'up', 'down', 'right', 'right', 'wait', 'right', 'right', 'wait', 'wait', 'wait', 'down', 'up', 'right', 'down', 'right', 'wait', 'wait', 'wait', 'wait', 'up', 'wait', 'up', 'right', 'right', 'wait', 'up', 'wait', 'up', 'left', 'down', 'wait', 'wait', 'up', 'right', 'right', 'right', 'right', 'down', 'right', 'wait', 'down', 'right', 'right', 'right', 'right', 'right', 'down', 'right', 'wait', 'wait', 'up', 'up', 'up', 'right', 'right', 'right', 'right', 'down', 'right', 'up', 'right', 'right', 'right', 'right', 'right', 'wait', 'up', 'wait', 'wait', 'wait', 'down', 'right', 'wait', 'wait', 'down', 'down', 'wait', 'down', 'right', 'wait', 'down', 'right', 'right', 'down', 'down', 'left', 'wait', 'right', 'wait', 'up', 'right', 'up', 'down', 'wait', 'down', 'down', 'right', 'right', 'right', 'right', 'up', 'wait', 'wait', 'wait', 'wait', 'up', 'up', 'right', 'right', 'wait', 'down', 'right', 'right', 'right', 'down', 'right', 'right', 'right', 'right', 'right', 'wait', 'left', 'right', 'down', 'right', 'wait', 'wait', 'right', 'up', 'right', 'right', 'up', 'down', 'right', 'down', 'right', 'right', 'down']

env = Environment(file_path="day24/input.txt")
# env.printout()
print()

for action in actions:
    # print(action)
    # print(hash(str(env.get_data())))
    env.step(action)
    # env.printout()
    # print()
    if env.check_status() <0:
        print("invalid")
    elif env.check_status()==1:
        print("done!")

    
    


done!
